In [1]:
from IPython.core.display import HTML
with open('../style.css') as f:
    css = f.read()
HTML(css)

# Alpha-Beta Pruning

This notebook implements [alpha-beta pruning](https://en.wikipedia.org/wiki/Alpha-beta_pruning). *Memoization techniques* are only added in a naive way since adding these techniques in a sophisticated way results in an algorithm that is quite complicated.

Effectively, this notebook is a *game solver* because it can be used to play various deterministic, zero-sum, two-person games with perfect information.  To this end, the implementation assumes that an external notebook defines a game and provides the following variables and functions:
* `gPlayers` is a list of length two.  The elements of this list are the 
  players.  It is assumed that the first element in this list represents 
  the computer, while the second element is the human player.  The computer
  always starts the game.
* `gStart` is the start state of the game.
* `next_states(state, player)` is a function that takes two arguments:
  - `state` is a state of the game.
  - `player` is the player whose turn it is to make a move.
  The function call `next_states(state, player)` returns the list
  of all states that can be reached by any move of `player`.
* `utility(state)` takes a `state` and a `player` as its arguments.
  If `state` is a *terminal state*, then the function returns the value
  that this `state` has for the player who starts the game.  Otherwise, the function returns `None`.
* `finished(state)` returns `True` if and only if `state` is a terminal state.
* `get_move(state)` displays the given state and asks the human player for
  her move.
* `final_msg(state)` informs the human player about the result of the game.
* `draw(state, canvas, value)` draws the given state on the canvas and informs
  the user about the `value` of this state. 
   
---

In order to have some variation in our game, we use random numbers to choose between optimal moves.

In [2]:
import random
random.seed(0)

The function `alphaBetaMax` takes three arguments:
- `State` is the current state of the game,
- `alpha` is a lower bound,
- `beta` is an upper bound.

The function `max_Value` returns the *value* that the given `State` has for the computer 
if both players play their optimal game.  This value is an element from the set $\{-1, 0, 1\}$.  
- If the first player can force a win, the return value is `1`.
- If the first player can at best force a draw, the return value is `0`.
- If the second player can force a win, the return value is `-1`.

Given that $\texttt{value}(s)$ is the value of the state $s$, the function `alphaBetaMax` satisfies the following specification:
- $\alpha \leq \texttt{maxValue}(s) \leq \beta \;\rightarrow\;\texttt{alphaBetaMax}(s, \alpha, \beta) = \texttt{maxValue}(s)$
- $\texttt{maxValue}(s) < \alpha \;\rightarrow\; \texttt{alphaBetaMax}(s, \alpha, \beta) \leq \alpha$
- $\beta < \texttt{maxValue}(s) \;\rightarrow\; \beta \leq \texttt{alphaBetaMax}(s, \alpha, \beta)$

Note that this specification does not define the function `alphaBetaMax` as there are many functions that satisfy 
this specification.  

In [3]:
def alphaBetaMax(State, alpha, beta):
    if finished(State):
        return utility(State)
    for ns in next_states(State, gPlayers[0]):
        value = alphaBetaMin(ns, alpha, beta)
        if value >= beta:
            return value
        alpha = max(alpha, value)
    return alpha

Given that $\texttt{minValue}(s)$ is the value of the state $s$, the function `alphaBetaMin` satisfies the following specification:
- $\alpha \leq \texttt{minValue}(s) \leq \beta \;\rightarrow\;\texttt{alphaBetaMin}(s, \alpha, \beta) = \texttt{minValue}(s)$
- $\texttt{minValue}(s) < \alpha \;\rightarrow\; \texttt{alphaBetaMin}(s, \alpha, \beta) \leq \alpha$
- $\beta < \texttt{minValue}(s) \;\rightarrow\; \beta \leq \texttt{alphaBetaMin}(s, \alpha, \beta)$

In [4]:
def alphaBetaMin(State, alpha, beta):
    if finished(State):
        return utility(State)
    for ns in next_states(State, gPlayers[1]):
        value = alphaBetaMax(ns, alpha, beta)
        if value <= alpha:
            return value
        beta = min(beta, value)
    return beta

In [5]:
import random
random.seed(1)

The function `best_move` takes one argument:
- `State` is the current state of the game.

The function `best_move` returns a pair of the form $(v, s)$ where $s$ is a state and $v$ is the value of this state.  The state $s$ is a state that is reached from `State` if `player` makes one of her optimal moves.  In order to have some variation in the game, the function randomly chooses any of the optimal moves.

In [6]:
def best_move(State):
    NS        = next_states(State, gPlayers[0])
    bestVal   = alphaBetaMax(State, -1, 1)
    print(f'The state {State} has value {bestVal}.')
    bestMoves = [s for s in NS if alphaBetaMin(s, -1, 1) == bestVal]
    bestState = random.choice(bestMoves)
    print(f'My move is: {bestState}')
    return bestVal, bestState

The next line is needed because we need the function `IPython.display.clear_output` to clear the output in a cell.

In [7]:
import IPython.display 

The function `play_game` plays the game that is specified externally via the functions `next_states` and `utility`.  The states of the game are drawn on the given `canvas`. 

In [8]:
def play_game(canvas):
    State = gStart
    while (True):
        val, State = best_move(State);
        draw(State, canvas, val)
        if finished(State):
            final_msg(State)
            return
        IPython.display.clear_output(wait=True)
        State = get_move(State)
        draw(State, canvas, '?')
        if finished(State):
            IPython.display.clear_output(wait=True)
            final_msg(State)
            break

In [10]:
%%capture
%run Nim.ipynb

In [11]:
#%%capture
#%run 2-Tic-Tac-Toe-Bitboard.ipynb

If we use *$\alpha$-$\beta$ pruning*, computing the value of the `Start` state of *tic-tac-toe* takes about 80 ms.  This should be compared to the more than 2 seconds that the *minimax algorithm* needs for the bit-board implementation of *tic-tac-toe*.

In [12]:
#%%time
#val = alphaBetaMax(gStart, -1, 1)
#val

## Naive Memoization

The implementation of `memoize` that is given below uses the fact that the functions `alphaBetaMax` and `alphaBetaMin` are never called with the same arguments.  Hence, we do not need to store the function `f`.

In [13]:
gCache = {}

def memoize(f):
    global gCache
    
    def f_memoized(*args):
        if args in gCache:
            return gCache[args]
        result = f(*args)
        gCache[args] = result
        return result
    
    return f_memoized

In [14]:
alphaBetaMax = memoize(alphaBetaMax)
alphaBetaMin = memoize(alphaBetaMin)

In [15]:
%%time
val = alphaBetaMax(gStart, -1, 1)
val

CPU times: user 10.4 ms, sys: 425 µs, total: 10.8 ms
Wall time: 10.7 ms


-1

When we check the size of the cache we realize that there are many states which we don't have to evaluate any more.

In [16]:
len(gCache)

710

Let's draw the board.

In [22]:
canvas = create_canvas()
draw(gStart, canvas, val)

Canvas()

Now its time to play.  In the input window that will pop up later, enter your move in the format "`row, count`".  Here `row` is the row that you choose.  The counting of the row  starts form 0. `count` is the number of matches that you want to take from the specified row. 

In [23]:
play_game(canvas)

You have won!
